# Electrolyzer Simulator — Notebooks de Figuras (IC)
Este notebook gera quatro gráficos: (1) I–V; (2) eficiência vs. temperatura; (3) distribuição de sobretensões; (4) dinâmica com degraus de corrente.
As equações implementadas: Nernst, Tafel, queda ôhmica e concentração.

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({
    "figure.dpi": 120,
    "font.size": 12,
    "axes.grid": True
})

R = 8.314462618  # J/mol/K
F = 96485.33212  # C/mol


In [ ]:
def nernst_voltage(T_K, p_H2=1.0, p_O2=1.0, p_H2O=1.0):
    V0 = 1.229
    V_T = V0 - 0.0009*(T_K - 298.15)
    return V_T + (R*T_K)/(2*F)*np.log((p_H2*(p_O2**0.5))/max(p_H2O, 1e-9))

def tafel_eta(i, i0, T_K, alpha=0.5):
    i = np.maximum(i, 1e-9)
    return (R*T_K)/(alpha*F)*np.log(i/np.maximum(i0,1e-12))

def ohmic_eta(i, ASR):
    return i*ASR

def concentration_eta(i, i_lim, T_K):
    eps = 1e-9
    ratio = np.clip(1 - i/np.maximum(i_lim, eps), eps, 1-1e-6)
    return (R*T_K)/(2*F)*np.log(1/ratio)

def v_cell(i, params, T_K):
    Vrev = nernst_voltage(T_K, params.get("p_H2",1), params.get("p_O2",1), params.get("p_H2O",1))
    eta_act = tafel_eta(i, params["i0"](T_K), T_K, params.get("alpha",0.5))
    eta_ohm = ohmic_eta(i, params["ASR"](T_K))
    eta_conc = concentration_eta(i, params["i_lim"](T_K), T_K)
    return Vrev + eta_act + eta_ohm + eta_conc, (Vrev, eta_act, eta_ohm, eta_conc)

def faradaic_efficiency(i):
    return 0.98 - 0.02*np.exp(-i/0.2)

def energy_efficiency(Vrev, Vcell, eta_F=0.98):
    return (Vrev/np.maximum(Vcell, 1e-9))*eta_F


In [ ]:
def arrhenius(val_25C, Ea_kJmol, T_K):
    Ea = Ea_kJmol*1000.0
    return val_25C*np.exp(-Ea/R*(1/T_K - 1/298.15))

def ASR_AWE(T_K):
    return max(0.20 - 0.0006*(T_K-298.15), 0.06)

def ASR_PEM(T_K):
    return max(0.15 - 0.0005*(T_K-298.15), 0.05)

def ASR_SOEC(T_K):
    return max(0.05 - 0.00025*(T_K-298.15), 0.01)

def i0_AWE(T_K):
    return arrhenius(1e-6, 25.0, T_K)

def i0_PEM(T_K):
    return arrhenius(5e-6, 22.0, T_K)

def i0_SOEC(T_K):
    return arrhenius(1e-3, 10.0, T_K)

def i_lim_AWE(T_K):
    return 2.8 + 0.002*(T_K-298.15)

def i_lim_PEM(T_K):
    return 2.5 + 0.0015*(T_K-298.15)

def i_lim_SOEC(T_K):
    return 3.0 + 0.002*(T_K-298.15)

PARAMS = {
    "AWE": {"ASR": ASR_AWE, "i0": i0_AWE, "i_lim": i_lim_AWE, "alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
    "PEM": {"ASR": ASR_PEM, "i0": i0_PEM, "i_lim": i_lim_PEM, "alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
    "SOEC": {"ASR": ASR_SOEC, "i0": i0_SOEC, "i_lim": i_lim_SOEC, "alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
}


## 1) Curvas de polarização (I–V) — AWE, PEM, SOEC

In [ ]:
T_C = 60.0
T_K = 273.15 + T_C
i_grid = np.linspace(0.2, 2.5, 150)

fig, ax = plt.subplots(figsize=(6,4))
for tech,label in [("AWE","Alkaline"), ("PEM","PEM"), ("SOEC","SOEC")]:
    Vcell,_ = v_cell(i_grid, PARAMS[tech], T_K)
    ax.plot(i_grid, Vcell, label=f"{label} eletrolizer")
ax.set_xlabel("Current density (A/cm$^2$)")
ax.set_ylabel("Cell Voltage (V)")
ax.legend()
plt.show()


## 2) Eficiência energética vs. temperatura — AWE, PEM, SOEC

In [ ]:
T_list = np.linspace(308.15, 353.15, 10)  # 35–80 ºC
i_op = 1.2
eff = {"AWE":[], "PEM":[], "SOEC":[]}

for T_K in T_list:
    for tech in eff.keys():
        Vcell,(Vrev, *_)= v_cell(i_op, PARAMS[tech], T_K)
        eff[tech].append(100*energy_efficiency(Vrev, Vcell, eta_F=faradaic_efficiency(i_op)))

fig, ax = plt.subplots(figsize=(6,4))
for tech,label in [("AWE","Alkaline"), ("PEM","PEM"), ("SOEC","SOEC")]:
    ax.plot(T_list-273.15, eff[tech], marker="o", label=f"{label} electrolyzer")
ax.set_xlabel("Temperatura (°C)")
ax.set_ylabel("Eficiência Energética (%)")
ax.legend()
plt.show()


## 3) Distribuição de sobretensões vs. temperatura — AWE

In [ ]:
T_list = np.linspace(298.15, 353.15, 12)  # 25–80 ºC
i_op = 1.5
fracs = []
for T_K in T_list:
    Vcell,(Vrev, eta_act, eta_ohm, eta_conc) = v_cell(i_op, PARAMS["AWE"], T_K)
    total_losses = max(eta_act + eta_ohm + eta_conc, 1e-9)
    fracs.append([100*eta_act/total_losses, 100*eta_ohm/total_losses, 100*eta_conc/total_losses])
fracs = np.array(fracs)

fig, ax = plt.subplots(figsize=(6,4))
ax.stackplot(T_list-273.15, fracs[:,0], fracs[:,1], fracs[:,2], labels=["Ativação","Ôhmica","Concentração"])
ax.set_xlabel("Temperatura (°C)")
ax.set_ylabel("Distribuição de Sobretensão (%)")
ax.legend(loc="upper right")
plt.show()


## 4) Simulação dinâmica (degraus de corrente) — AWE

In [ ]:
t = np.arange(0, 60*60+1, 10)  # 0–60 min em 10 s
steps_Acm2 = [0.7, 0.6, 1.0, 0.9, 1.2, 1.1, 1.4]
t_step = np.linspace(5, 55, len(steps_Acm2))*60  # s
i_series = np.ones_like(t, dtype=float)*steps_Acm2[0]
for k, tk in enumerate(t_step[1:], start=1):
    i_series[t>=tk] = steps_Acm2[k]

T_base = 333.15  # 60 ºC
tau = 15*60
T_series = np.zeros_like(t, dtype=float)
T_series[0] = T_base
for k in range(1, len(t)):
    T_target = T_base + 4.0*(i_series[k]-np.mean(steps_Acm2))
    T_series[k] = T_series[k-1] + ((T_target - T_series[k-1]) * (t[k]-t[k-1]) / tau)

eff_series = np.zeros_like(t, dtype=float)
for k, (Ti, ik) in enumerate(zip(T_series, i_series)):
    Vcell,(Vrev, *_)= v_cell(max(ik,1e-6), PARAMS["AWE"], Ti)
    eff_series[k] = 100*energy_efficiency(Vrev, Vcell, eta_F=faradaic_efficiency(max(ik,1e-6)))

fig, ax1 = plt.subplots(figsize=(6,4))
ax1.plot(t/60.0, eff_series, label="Eficiência Energética (%)")
ax1.set_xlabel("Tempo (min)")
ax1.set_ylabel("Eficiência Energética (%)")
ax2 = ax1.twinx()
ax2.plot(t/60.0, i_series, ls="--", label="Densidade de Corrente (A/cm$^2$)")
ax2.set_ylabel("Densidade de Corrente (A/cm$^2$)")
fig.legend(loc="upper right", bbox_to_anchor=(0.9,0.9))
plt.show()
